In [31]:
import os
from sklearn.model_selection import train_test_split
import skimage
from skimage import io, color
import numpy as np
from sklearn.neighbors  import KNeighborsClassifier
import pandas as pd
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize, downscale_local_mean
from sklearn.metrics import accuracy_score, confusion_matrix
import cv2

In [32]:
Data_path_test = '..\Projet IA\datasets'
Im_test_path = ['..\Projet IA\Images_Test\Test_face19.jpg']

In [33]:
# Fonction pour lire les images de la base
# images = vecteurs contenant les paths desimages 
# labels = vecteur contenant les éthiquettes 
def Lire_data(data_path):
    images=[]
    labels=[]
    for classe_name in os.listdir(data_path):
        classe_path = os.path.join(data_path, classe_name)
        for file in os.listdir(classe_path):
            image_path = os.path.join(classe_path, file)
            images.append(image_path)
            labels.append(classe_name)
    return images, labels

In [34]:
# Transformation numérique des données
def Traitement_Images(Images_Path): 
    X_=[]
    for image_path in Images_Path:
        I=io.imread(image_path)
        I=color.rgb2gray(I)
        I=resize(I,(100, 100))
        x=I.flatten()
        X_.append(x)
    X_=np.array(X_)
    return X_

In [35]:
X,Y = Lire_data(Data_path_test)

In [36]:
X_train, X_test, y_train, y_test=train_test_split(X,Y, train_size=0.68, stratify=Y)

In [37]:
X_train_=Traitement_Images(X_train)

In [38]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_,y_train)

KNeighborsClassifier()

In [39]:
X_test_ = Traitement_Images(X_test)

y_test_predicted=knn.predict(X_test_)

In [40]:
print("accuracy =  %.2f " % (accuracy_score(y_test, y_test_predicted)*100))
M=confusion_matrix(y_test, y_test_predicted)
print('matrice de confusion = \n',pd.DataFrame(M,columns=knn.classes_,index=knn.classes_))

accuracy =  84.38 
matrice de confusion = 
           Hechmi  Inconnue  Marwen
Hechmi        31         0       1
Inconnue       5        24       3
Marwen         6         0      26


In [41]:
Im_test_ = Traitement_Images(Im_test_path)

In [42]:
Im_test_predicted=knn.predict(Im_test_.reshape(1,-1))
print('Prédiction N_Image = \n',Im_test_predicted)

Prédiction N_Image = 
 ['Hechmi']


In [43]:
cap = cv2.VideoCapture(0)

classifier = cv2.CascadeClassifier("..\Projet IA\haarcascade_frontalface_default.xml")
f_list = []

while True:

	ret, frame = cap.read()

	gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

	# Capture des visage avec webcam
	faces = classifier.detectMultiScale(gray, 1.5, 5)

	X_test = []

	
	# traitement numérique des visages capturés
	for face in faces:
		x, y, w, h = face
		im_face = gray[y:y + h, x:x + w]
		im_face = resize(im_face, (100, 100))
		X_test.append(im_face.flatten())


	if len(faces)>0:
		response = knn.predict(np.array(X_test))
		# prediction du résultat avec KNN

		for i, face in enumerate(faces):
			x, y, w, h = face

			# dessiner un rectangle sur le visage détecté
			cv2.rectangle(frame, (x, y), (x + w, y + h),
										(255, 0, 0), 3)

			# ajouter un nom prédit pour le visage
			cv2.putText(frame, response[i], (x-50, y-50),
							cv2.FONT_HERSHEY_DUPLEX, 2,
										(0, 255, 0), 3)
	
	cv2.imshow("full", frame)

	key = cv2.waitKey(1)

	if key & 0xFF == ord("q") :
		break

cap.release()
cv2.destroyAllWindows()
